In [1]:
!pip install order_of_magnitude
!pip install netCDF4

In [2]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("drive/My Drive/ERA5_Dataset")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
import pandas as pd
import netCDF4 as nc
import util

In [4]:
dates = util.dates_list('1980/01/01', '2022/12/31')
print(len(dates))
N_Samples = len(dates)

15706


## Target Dataframe

In [5]:
df_target = pd.read_csv('./csv/target_NoOutliers_complete.csv')

lats = df_target['LAT'].values
lons = df_target['LON'].values

df_target

,DATE,ID,LAT,LON,S.IndAll
0,1980-01-01,NaN,-18.50,80.50,0.00
1,1980-01-02,NaN,-30.00,31.25,0.00
2,1980-01-03,NaN,-4.00,86.50,0.00
3,1980-01-04,NaN,-29.50,32.75,0.00
4,1980-01-05,NaN,-4.75,89.25,0.00
...,...,...,...,...,...
15422,2022-03-23,392.0,-13.75,76.75,0.50
15423,2022-03-24,392.0,-14.25,74.25,1.00
15424,2022-03-25,392.0,-13.50,73.25,1.00
15425,2022-03-26,392.0,-15.00,74.00,1.00


In [6]:
N_dates = df_target.shape[0]

## Temperature At Levels 550hPa_300hPa_200hPa

In [7]:
ds_temperature = nc.Dataset('../ERA5_Dataset/nc_original_datasets/temperature_200_300_550_hPa.nc')
#ds_temperature = nc.Dataset('../ERA5_Dataset/nc_original_datasets/temperature_1000hPa.nc')

In [8]:
latitude_arr = ds_temperature['latitude'][:]
longitude_arr = ds_temperature['longitude'][240:]

print(latitude_arr.shape, longitude_arr.shape)

(121,) (241,)


In [9]:
print((latitude_arr[0], latitude_arr[30]), (longitude_arr[0], longitude_arr[60]))
print((latitude_arr[0], latitude_arr[30]), (longitude_arr[60], longitude_arr[120]))
print((latitude_arr[0], latitude_arr[30]), (longitude_arr[120], longitude_arr[180]))
print((latitude_arr[0], latitude_arr[30]), (longitude_arr[180], longitude_arr[240]))
print('-------------------------------------------------------')
print((latitude_arr[30], latitude_arr[60]), (longitude_arr[0], longitude_arr[60]))
print((latitude_arr[30], latitude_arr[60]), (longitude_arr[60], longitude_arr[120]))
print((latitude_arr[30], latitude_arr[60]), (longitude_arr[120], longitude_arr[180]))
print((latitude_arr[30], latitude_arr[60]), (longitude_arr[180], longitude_arr[240]))
print('-------------------------------------------------------')
print((latitude_arr[60], latitude_arr[90]), (longitude_arr[0], longitude_arr[60]))
print((latitude_arr[60], latitude_arr[90]), (longitude_arr[60], longitude_arr[120]))
print((latitude_arr[60], latitude_arr[90]), (longitude_arr[120], longitude_arr[180]))
print((latitude_arr[60], latitude_arr[90]), (longitude_arr[180], longitude_arr[240]))
print('-------------------------------------------------------')
print((latitude_arr[90], latitude_arr[120]), (longitude_arr[0], longitude_arr[60]))
print((latitude_arr[90], latitude_arr[120]), (longitude_arr[60], longitude_arr[120]))
print((latitude_arr[90], latitude_arr[120]), (longitude_arr[120], longitude_arr[180]))
print((latitude_arr[90], latitude_arr[120]), (longitude_arr[180], longitude_arr[240]))

(0.0, -7.5) (30.0, 45.0)
(0.0, -7.5) (45.0, 60.0)
(0.0, -7.5) (60.0, 75.0)
(0.0, -7.5) (75.0, 90.0)
-------------------------------------------------------
(-7.5, -15.0) (30.0, 45.0)
(-7.5, -15.0) (45.0, 60.0)
(-7.5, -15.0) (60.0, 75.0)
(-7.5, -15.0) (75.0, 90.0)
-------------------------------------------------------
(-15.0, -22.5) (30.0, 45.0)
(-15.0, -22.5) (45.0, 60.0)
(-15.0, -22.5) (60.0, 75.0)
(-15.0, -22.5) (75.0, 90.0)
-------------------------------------------------------
(-22.5, -30.0) (30.0, 45.0)
(-22.5, -30.0) (45.0, 60.0)
(-22.5, -30.0) (60.0, 75.0)
(-22.5, -30.0) (75.0, 90.0)


In [10]:
ds_temperature['t']

<class 'netCDF4._netCDF4.Variable'>
int16 t(time, level, latitude, longitude)
    scale_factor: 0.0012095716808617652
    add_offset: 245.07787879570253
    _FillValue: -32767
    missing_value: -32767
    units: K
    long_name: Temperature
    standard_name: air_temperature
unlimited dimensions: 
current shape = (15706, 3, 121, 481)
filling on

In [11]:
temperature = ds_temperature['t'][:N_dates, 0, :, 240:] # 200hPa
#temperature = ds_temperature['t'][:N_dates, 1, :, 240:] # 300hPa
#temperature = ds_temperature['t'][:N_dates, 2, :, 240:] # 550hPa
#temperature = ds_temperature['t'][:N_dates, :, :] # 1000hPa
print(temperature.shape)

t_zones = []
t_zones.append(temperature[:N_dates, 0:30, 0:60])
t_zones.append(temperature[:N_dates, 0:30, 60:120])
t_zones.append(temperature[:N_dates, 0:30, 120:180])
t_zones.append(temperature[:N_dates, 0:30, 180:])

t_zones.append(temperature[:N_dates, 30:60, 0:60])
t_zones.append(temperature[:N_dates, 30:60, 60:120])
t_zones.append(temperature[:N_dates, 30:60, 120:180])
t_zones.append(temperature[:N_dates, 30:60, 180:])

t_zones.append(temperature[:N_dates, 60:90, 0:60])
t_zones.append(temperature[:N_dates, 60:90, 60:120])
t_zones.append(temperature[:N_dates, 60:90, 120:180])
t_zones.append(temperature[:N_dates, 60:90, 180:])

t_zones.append(temperature[:N_dates, 90:, 0:60])
t_zones.append(temperature[:N_dates, 90:, 60:120])
t_zones.append(temperature[:N_dates, 90:, 120:180])
t_zones.append(temperature[:N_dates, 90:, 180:])

print(len(t_zones))

(15427, 121, 241)
16


In [22]:
for i in range(16):
  print(t_zones[i].shape)

(15427, 30, 60)
(15427, 30, 60)
(15427, 30, 60)
(15427, 30, 61)
(15427, 30, 60)
(15427, 30, 60)
(15427, 30, 60)
(15427, 30, 61)
(15427, 30, 60)
(15427, 30, 60)
(15427, 30, 60)
(15427, 30, 61)
(15427, 31, 60)
(15427, 31, 60)
(15427, 31, 60)
(15427, 31, 61)


In [13]:
mean_zones = []
std_zones = []

for zone in range(16):
  means = []
  stds = []
  for date_idx in range(N_dates):
    temp_mean = np.mean(t_zones[zone][date_idx])
    temp_std = np.std(t_zones[zone][date_idx])
    means.append(temp_mean)
    stds.append(temp_std)
  mean_zones.append(means)
  std_zones.append(stds)

In [14]:
mean_std_zones = np.column_stack((mean_zones[0], std_zones[0],
                                  mean_zones[1], std_zones[1],
                                  mean_zones[2], std_zones[2],
                                  mean_zones[3], std_zones[3],
                                  mean_zones[4], std_zones[4],
                                  mean_zones[5], std_zones[5],
                                  mean_zones[6], std_zones[6],
                                  mean_zones[7], std_zones[7],
                                  mean_zones[8], std_zones[8],
                                  mean_zones[9], std_zones[9],
                                  mean_zones[10], std_zones[10],
                                  mean_zones[11], std_zones[11],
                                  mean_zones[12], std_zones[12],
                                  mean_zones[13], std_zones[13],
                                  mean_zones[14], std_zones[14],
                                  mean_zones[15], std_zones[15]))

# Save Datset

In [15]:
target_att = ['DATE', 'S.IndAll']
mean_std_zones_att = ['Mean_Z1', 'Std_Z1',
                      'Mean_Z2', 'Std_Z2',
                      'Mean_Z3', 'Std_Z3',
                      'Mean_Z4', 'Std_Z4',
                      'Mean_Z5', 'Std_Z5',
                      'Mean_Z6', 'Std_Z6',
                      'Mean_Z7', 'Std_Z7',
                      'Mean_Z8', 'Std_Z8',
                      'Mean_Z9', 'Std_Z9',
                      'Mean_Z10', 'Std_Z10',
                      'Mean_Z11', 'Std_Z11',
                      'Mean_Z12', 'Std_Z12',
                      'Mean_Z13', 'Std_Z13',
                      'Mean_Z14', 'Std_Z14',
                      'Mean_Z15', 'Std_Z15',
                      'Mean_Z16', 'Std_Z16']

target = df_target[target_att].values
final_array = np.hstack((target, mean_std_zones))

df_complete = pd.DataFrame(final_array, columns=target_att+mean_std_zones_att)
df_complete[11:]

,DATE,S.IndAll,Mean_Z1,Std_Z1,Mean_Z2,Std_Z2,Mean_Z3,Std_Z3,Mean_Z4,Std_Z4,...,Mean_Z12,Std_Z12,Mean_Z13,Std_Z13,Mean_Z14,Std_Z14,Mean_Z15,Std_Z15,Mean_Z16,Std_Z16
11,1980-01-12,0.0,220.031599,0.35544,219.564555,0.432616,219.122519,0.520932,218.757787,0.470001,...,219.449173,0.449474,220.685517,1.285497,220.680312,0.437651,219.593552,0.643608,220.35704,1.874089
12,1980-01-13,0.0,219.336468,0.47821,219.540437,0.278405,219.549282,0.609918,219.376219,0.347244,...,219.527131,0.341065,221.580113,0.948411,221.262653,0.469232,219.820347,0.532702,217.700147,1.357856
13,1980-01-14,0.0,218.708478,0.383002,219.279513,0.491183,219.225647,0.522359,219.108612,0.544596,...,219.241955,0.392216,219.646813,1.791283,220.945418,0.309651,219.729743,0.830292,216.629936,1.713481
14,1980-01-15,0.0,218.71562,0.626439,219.245073,0.619718,219.120877,0.530998,219.49698,0.475967,...,219.309764,0.6597,218.392631,1.974118,219.925217,0.578542,219.742059,0.311113,216.458852,2.471615
15,1980-01-16,0.0,219.622547,0.556058,219.3416,0.503601,218.858194,0.339596,218.670707,0.427064,...,219.594137,0.442953,218.227055,1.256291,219.43745,0.387647,219.200716,0.670142,218.483882,0.975346
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15422,2022-03-23,0.5,219.319951,0.324266,218.876166,0.241067,219.793181,0.597253,219.976111,0.482109,...,221.214912,0.456294,221.077486,0.501165,220.386891,0.448278,220.14083,0.393765,220.021914,1.438795
15423,2022-03-24,1.0,218.772251,0.66617,218.763867,0.583032,220.332447,0.654425,220.317933,0.778707,...,221.279832,0.45771,221.296659,0.632314,220.462278,0.293922,221.257688,0.628521,221.54277,0.971513
15424,2022-03-25,1.0,218.77571,0.486007,218.526857,0.422838,219.21839,0.973833,220.286845,0.728068,...,221.702256,0.605113,220.763351,0.845368,221.10935,0.274504,222.191774,0.892029,221.841748,0.722973
15425,2022-03-26,1.0,219.746559,0.427021,219.507637,0.293396,219.555309,0.743522,219.943117,0.452705,...,221.913019,0.780803,219.56025,0.368298,220.463773,0.823968,223.522317,1.100126,222.081735,0.754599


# Rollback 10 days

In [16]:
def attribute_stepped(atts, max_steps):
  att_list = []
  for att in atts:
    for i in range(max_steps):
      att_list.append('{}-{}'.format(att,i))
  return att_list

In [17]:
def mean_std_atts_zones(zones_n):
  new_att_list = []
  for i in range(zones_n):
    new_att_list.append('Mean_Z{}'.format(i+1))
    new_att_list.append('Std_Z{}'.format(i+1))
  return new_att_list

print(mean_std_atts_zones(16))

['Mean_Z1', 'Std_Z1', 'Mean_Z2', 'Std_Z2', 'Mean_Z3', 'Std_Z3', 'Mean_Z4', 'Std_Z4', 'Mean_Z5', 'Std_Z5', 'Mean_Z6', 'Std_Z6', 'Mean_Z7', 'Std_Z7', 'Mean_Z8', 'Std_Z8', 'Mean_Z9', 'Std_Z9', 'Mean_Z10', 'Std_Z10', 'Mean_Z11', 'Std_Z11', 'Mean_Z12', 'Std_Z12', 'Mean_Z13', 'Std_Z13', 'Mean_Z14', 'Std_Z14', 'Mean_Z15', 'Std_Z15', 'Mean_Z16', 'Std_Z16']


In [18]:
max_steps = 11
labels_to_rewind = mean_std_atts_zones(16)
labels_stepped = attribute_stepped(labels_to_rewind, max_steps)
print(labels_stepped)

stepped_mean_zones = []
stepped_std_zones = []

for zone in range(1, 17):
  mean_press_10d = np.zeros((N_dates-max_steps, max_steps))
  std_press_10d = np.zeros((N_dates-max_steps, max_steps))

  mean_press = df_complete['Mean_Z{}'.format(zone)]
  std_press = df_complete['Std_Z{}'.format(zone)]

  for day in range(max_steps, N_dates):
    for steps in range(max_steps):
      mean_press_10d[day-max_steps, steps] = mean_press[day-steps]
      std_press_10d[day-max_steps, steps] = std_press[day-steps]
  print(mean_press_10d.shape)
  stepped_mean_zones.append(mean_press_10d)
  stepped_std_zones.append(std_press_10d)

['Mean_Z1-0', 'Mean_Z1-1', 'Mean_Z1-2', 'Mean_Z1-3', 'Mean_Z1-4', 'Mean_Z1-5', 'Mean_Z1-6', 'Mean_Z1-7', 'Mean_Z1-8', 'Mean_Z1-9', 'Mean_Z1-10', 'Std_Z1-0', 'Std_Z1-1', 'Std_Z1-2', 'Std_Z1-3', 'Std_Z1-4', 'Std_Z1-5', 'Std_Z1-6', 'Std_Z1-7', 'Std_Z1-8', 'Std_Z1-9', 'Std_Z1-10', 'Mean_Z2-0', 'Mean_Z2-1', 'Mean_Z2-2', 'Mean_Z2-3', 'Mean_Z2-4', 'Mean_Z2-5', 'Mean_Z2-6', 'Mean_Z2-7', 'Mean_Z2-8', 'Mean_Z2-9', 'Mean_Z2-10', 'Std_Z2-0', 'Std_Z2-1', 'Std_Z2-2', 'Std_Z2-3', 'Std_Z2-4', 'Std_Z2-5', 'Std_Z2-6', 'Std_Z2-7', 'Std_Z2-8', 'Std_Z2-9', 'Std_Z2-10', 'Mean_Z3-0', 'Mean_Z3-1', 'Mean_Z3-2', 'Mean_Z3-3', 'Mean_Z3-4', 'Mean_Z3-5', 'Mean_Z3-6', 'Mean_Z3-7', 'Mean_Z3-8', 'Mean_Z3-9', 'Mean_Z3-10', 'Std_Z3-0', 'Std_Z3-1', 'Std_Z3-2', 'Std_Z3-3', 'Std_Z3-4', 'Std_Z3-5', 'Std_Z3-6', 'Std_Z3-7', 'Std_Z3-8', 'Std_Z3-9', 'Std_Z3-10', 'Mean_Z4-0', 'Mean_Z4-1', 'Mean_Z4-2', 'Mean_Z4-3', 'Mean_Z4-4', 'Mean_Z4-5', 'Mean_Z4-6', 'Mean_Z4-7', 'Mean_Z4-8', 'Mean_Z4-9', 'Mean_Z4-10', 'Std_Z4-0', 'Std_Z4-1', 

In [19]:
final_array = np.hstack((target[max_steps:],
                         stepped_mean_zones[0], stepped_std_zones[0],
                         stepped_mean_zones[1], stepped_std_zones[1],
                         stepped_mean_zones[2], stepped_std_zones[2],
                         stepped_mean_zones[3], stepped_std_zones[3],
                         stepped_mean_zones[4], stepped_std_zones[4],
                         stepped_mean_zones[5], stepped_std_zones[5],
                         stepped_mean_zones[6], stepped_std_zones[6],
                         stepped_mean_zones[7], stepped_std_zones[7],
                         stepped_mean_zones[8], stepped_std_zones[8],
                         stepped_mean_zones[9], stepped_std_zones[9],
                         stepped_mean_zones[10], stepped_std_zones[10],
                         stepped_mean_zones[11], stepped_std_zones[11],
                         stepped_mean_zones[12], stepped_std_zones[12],
                         stepped_mean_zones[13], stepped_std_zones[13],
                         stepped_mean_zones[14], stepped_std_zones[14],
                         stepped_mean_zones[15], stepped_std_zones[15],
                         ))
df_stepped = pd.DataFrame(final_array , columns = target_att + labels_stepped)
df_stepped

,DATE,S.IndAll,Mean_Z1-0,Mean_Z1-1,Mean_Z1-2,Mean_Z1-3,Mean_Z1-4,Mean_Z1-5,Mean_Z1-6,Mean_Z1-7,...,Std_Z16-1,Std_Z16-2,Std_Z16-3,Std_Z16-4,Std_Z16-5,Std_Z16-6,Std_Z16-7,Std_Z16-8,Std_Z16-9,Std_Z16-10
0,1980-01-12,0.0,220.031599,220.107611,219.506949,219.082111,219.189421,219.37268,219.227063,218.873189,...,0.686731,0.884226,0.994397,1.095202,1.183172,1.36814,1.528161,1.09029,0.719526,0.686094
1,1980-01-13,0.0,219.336468,220.031599,220.107611,219.506949,219.082111,219.189421,219.37268,219.227063,...,1.874089,0.686731,0.884226,0.994397,1.095202,1.183172,1.36814,1.528161,1.09029,0.719526
2,1980-01-14,0.0,218.708478,219.336468,220.031599,220.107611,219.506949,219.082111,219.189421,219.37268,...,1.357856,1.874089,0.686731,0.884226,0.994397,1.095202,1.183172,1.36814,1.528161,1.09029
3,1980-01-15,0.0,218.71562,218.708478,219.336468,220.031599,220.107611,219.506949,219.082111,219.189421,...,1.713481,1.357856,1.874089,0.686731,0.884226,0.994397,1.095202,1.183172,1.36814,1.528161
4,1980-01-16,0.0,219.622547,218.71562,218.708478,219.336468,220.031599,220.107611,219.506949,219.082111,...,2.471615,1.713481,1.357856,1.874089,0.686731,0.884226,0.994397,1.095202,1.183172,1.36814
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15411,2022-03-23,0.5,219.319951,219.167548,219.232487,219.910158,219.672019,219.828496,220.377924,220.07579,...,1.093564,1.462673,1.735088,1.440886,1.241267,1.17032,1.668688,0.901483,0.557791,0.794303
15412,2022-03-24,1.0,218.772251,219.319951,219.167548,219.232487,219.910158,219.672019,219.828496,220.377924,...,1.438795,1.093564,1.462673,1.735088,1.440886,1.241267,1.17032,1.668688,0.901483,0.557791
15413,2022-03-25,1.0,218.77571,218.772251,219.319951,219.167548,219.232487,219.910158,219.672019,219.828496,...,0.971513,1.438795,1.093564,1.462673,1.735088,1.440886,1.241267,1.17032,1.668688,0.901483
15414,2022-03-26,1.0,219.746559,218.77571,218.772251,219.319951,219.167548,219.232487,219.910158,219.672019,...,0.722973,0.971513,1.438795,1.093564,1.462673,1.735088,1.440886,1.241267,1.17032,1.668688


In [20]:
df_stepped.columns[13::22]

Index(['Std_Z1-0', 'Std_Z2-0', 'Std_Z3-0', 'Std_Z4-0', 'Std_Z5-0', 'Std_Z6-0',
       'Std_Z7-0', 'Std_Z8-0', 'Std_Z9-0', 'Std_Z10-0', 'Std_Z11-0',
       'Std_Z12-0', 'Std_Z13-0', 'Std_Z14-0', 'Std_Z15-0', 'Std_Z16-0'],
      dtype='object')

In [21]:
df_stepped.to_csv('./csv/16_zones/temperature_200hPa_10D.csv', index=False)